In [1]:
import os
import numpy as np
import numba as  nb
import pandas as pd


import plotly.graph_objects as go
from plotly.graph_objects import Figure
from plotly.graph_objects import Scatter

from  sxo.util.runtime.timeseries import RedisTs

In [2]:
os.environ["REDIS_PASS"] = "boss"

In [5]:
ts_bid = RedisTs('ts_bid_37374')
ts_ask = RedisTs('ts_ask_37374')
ts_bsz = RedisTs('ts_bsz_37374')
ts_asz = RedisTs('ts_asz_37374')
bid = ts_bid.get_range(name='bid')
ask = ts_ask.get_range(name='ask')
bsz = ts_bid.get_range(name='bsz')
asz = ts_ask.get_range(name='asz')
bid_ask = bid.join(ask, how='inner').join(bsz, how='inner').join(asz, how='inner').reset_index()
bid_ask['t'] = bid_ask['t'].values.astype('datetime64[ms]')


In [8]:

df_plt = bid_ask.copy()[bid_ask.t > np.datetime64('2023-06-21 16:00:00')]
m = 15
fig = Figure(layout=go.Layout(height=500, margin={'t':m,'b':m,'l':m,'r':m}))
fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['ask'], name='ask', mode="markers", marker={'size':2},showlegend=False))
fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['bid'], name='bid', mode="markers", marker={'size':2},showlegend=False))
fig.add_traces(Scatter(x=df_plt['t'], y=(df_plt['bid'] + df_plt['ask'])/2, name='mid', mode="markers", marker={'size':2, 'color':'black'},showlegend=False))
fig.show()

In [10]:
# def make_bin(x):
#     df = pd.Series()
#     t = x['t'].values
#     bid, ask, bsz, asz = x.bid.values, x.ask.values, x.bsz.values, x.asz.values
#     bin_end = x['bin_end'].values[0]
#     mid  = (bid + ask)/2.0
#     dt = np.diff(np.append(t, bin_end))


#     df['bin_end'] = bin_end 
#     df['o'] = mid[0]
#     df['h'] = np.max(mid)
#     df['l'] = np.min(mid)
#     df['c'] = mid[-1]
#     df['twap'] = np.sum(mid*dt)/np.sum(dt)
#     df['mean'] = np.mean(mid)

#     return df

# def gen_binned_data(ticks:pd.DataFrame,
#                     bin_size_sec:int,
#                     time_col:str = "t",
#                     ) -> pd.DataFrame:

#     # this is needed for twaps to work correctlly, otherwise we see downward bias in twaps
#     ticks[time_col] = ticks[time_col].values.astype('datetime64[ns]')

#     ticks['bin_start'] = ticks[time_col].values.astype(f'datetime64[{bin_size_sec}s]').astype('datetime64[s]')
#     ticks['bin_end'] = ticks['bin_start'].values + np.timedelta64(bin_size_sec, 's')

#     padded = ticks.groupby(['bin_start'], as_index=False).apply(make_bin).reset_index()
#     return padded

# subset = bid_ask[bid_ask.t > np.datetime64('2023-06-19 15:23:00')].copy()
# x = gen_binned_data(subset, 60)
# x

In [97]:
@nb.jit(nopython=True)
def insert_double(a:np.array, idx, v:np.float64) -> np.array:
    N = len(a)
    if idx < 0:
        raise Exception("Insertion index must be 0 or greater.")
    if idx > N:
        raise Exception("Trying to insert beyond the actual array ")
    
    tgt = np.full(N+1, np.nan)
    for i in range (0, idx):
        tgt[i] = -1
    tgt[idx] = v
    for i in range (idx, N):
        tgt[i+1] = -2
    return tgt


@nb.jit(nopython=True)
def xxx(a:np.array) -> np.array:
    for i in range(len(a)):
        if i%2 == 0:
            np.insert(a, i, -2)
    return a

In [31]:
@nb.jit(nopython=True)
def do_a_manual(t:np.array, bid:np.array, ask:np.array, bin_idx:np.array, ) -> np.array:
    n_ticks, n_bins = len(t), len(np.unique(bin_idx))
    i, j = 0, 0
    bin_index = np.full(n_bins, np.nan)
    n_samples = np.full(n_bins, 0)
    while j < n_ticks:
        bin_index = bin_idx[i]
        while bin_idx[i] == bin_idx[j]:
            j += 1
        i = j
        
    
    return bin_index

#pad_on_bin_bopundary
def foo(ticks:pd.DataFrame,
        bin_size_sec:int,
        time_col:str = "t",
        ) -> pd.DataFrame:

    bin_start = ticks[time_col].values.astype(f'datetime64[{bin_size_sec}s]').astype('datetime64[s]')
    bin_index = do_a_manual(ticks.t.values, ticks.bid.values, ticks.ask.values, bin_start)
    return pd.DataFrame({'bin_start':bin_index})

foo(bid_ask.copy(), 60)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Cannot unify array(float64, 1d, C) and datetime64[s] for 'bin_index.2', defined at /tmp/ipykernel_14304/3429096403.py (14)

File "../../../../tmp/ipykernel_14304/3429096403.py", line 14:
<source missing, REPL/exec in use?>

During: typing of assignment at /tmp/ipykernel_14304/3429096403.py (14)

File "../../../../tmp/ipykernel_14304/3429096403.py", line 14:
<source missing, REPL/exec in use?>


In [90]:
annotated['bin_start'].values

array(['2023-06-06T19:22:00.000000000', '2023-06-06T19:22:00.000000000',
       '2023-06-06T19:22:00.000000000', ...,
       '2023-06-20T18:03:00.000000000', '2023-06-20T18:03:00.000000000',
       '2023-06-20T18:03:00.000000000'], dtype='datetime64[ns]')

,t,bid,ask,bsz,asz
0,2023-06-06 19:22:24.585,1.16200,1.16237,1.16200,1.16237
1,2023-06-06 19:22:25.612,1.16201,1.16238,1.16201,1.16238
2,2023-06-06 19:22:31.087,1.16202,1.16239,1.16202,1.16239
3,2023-06-06 19:22:32.525,1.16201,1.16238,1.16201,1.16238
4,2023-06-06 19:22:34.189,1.16200,1.16237,1.16200,1.16237
...,...,...,...,...,...
196263,2023-06-20 06:48:39.465,1.16954,1.16991,1.16954,1.16991
196264,2023-06-20 06:48:40.766,1.16952,1.16989,1.16952,1.16989
196265,2023-06-20 06:48:41.801,1.16950,1.16987,1.16950,1.16987
196266,2023-06-20 06:48:42.850,1.16950,1.16987,1.16950,1.16987
